In [1]:
def select_encoding_and_bases(length):
    bitstring = ""
    bases = "" 
    for i in range(length):
        # We use the function getrandbits to get either a 0 or 1 randomly,
        # The "1" in the function argument is the number of bits to be generated
        bitstring += (str(getrandbits(1)))
        # 0 means encode in the (0,1) basis and 1 means encode in the (+,-) basis
        bases += (str(getrandbits(1)))
    
    # return the string of bits and the list of bases they should be encoded in
    return bitstring, bases

In [2]:
def measure(bases, encoded_qubits, backend):
    bitstring = ''
    for i in range(len(encoded_qubits)):
        qc = encoded_qubits[i]
        
        if bases[i] == "0":
            # 0 means we want to measure in Z basis
            qc.measure(0,0)

        elif bases[i] == "1":
            # 1 means we want to measure in X basis
            qc.h(0)
            qc.measure(0,0)
        
        # Now that the measurements have been added to the circuit, let's run them.
        job = q.execute(qc, backend=backend, shots = 1) # increase shots if running on hardware
        results = job.result()
        counts = results.get_counts()
        measured_bit = max(counts, key=counts.get)

        # Append measured bit to Bob's measured bitstring
        bitstring += measured_bit 
        
    return bitstring

In [3]:
def encode(bitstring, bases):
    encoded_qubits = []
    for i in range(len(bitstring)):
        # create a brand new quantum circuit called qc. Remember that the qubit will be in state |0> by default
        qc = q.QuantumCircuit(1,1)

        if bases[i] == "0":
            # 0 Means we are encoding in the z basis
            if bitstring[i] == "0":
                # We want to encode a |0> state, as states are intialized
                # in |0> by default we don't need to add anything here
                pass
            
            elif bitstring[i] == "1":
                # We want to encode a |1> state
                # We apply an X gate to generate |1>
                qc.x(0)
                
        elif bases[i] == "1":
            # 1 Means we are encoding in the x basis
            if bitstring[i] == "0":
                # We apply an H gate to generate |+>
                qc.h(0)
            elif bitstring[i] == "1":
                # We apply an X and an H gate to generate |->
                qc.x(0)
                qc.h(0)
            
        # add this quantum circuit to the list of encoded_qubits
        encoded_qubits.append(qc)
        
    return encoded_qubits

In [4]:
def bob_compare_bases(alices_bases, bobs_bases):
    indices = []
    
    for i in range(len(alices_bases)):
        if alices_bases[i] == bobs_bases[i]:
            indices.append(i)
    return indices

In [5]:
def construct_key_from_indices(bitstring, indices):
    key = ''
    for idx in indices:
        # For the indices where bases match, the bitstring bit is added to the key
        key = key + bitstring[idx] 
    return key

In [7]:
# QKD Algorithm with no evedropping¶

!sudo pip install qiskit
from random import getrandbits
import qiskit as q

     |████████████████████████████████| 6.0MB 7.3MB/s 
     |████████████████████████████████| 17.9MB 232kB/s 
     |████████████████████████████████| 204kB 54.9MB/s 
     |████████████████████████████████| 215kB 53.3MB/s 
     |████████████████████████████████| 2.1MB 50.7MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 1.0MB 43.1MB/s 
     |████████████████████████████████| 194kB 43.2MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 614kB 40.4MB/s 
     |████████████████████████████████| 3.2MB 49.7MB/s 
     |████████████████████████████████| 5.5MB 48.3MB/s 
  Created wheel for qiskit: filename=qiskit-0.25.1-cp37-none-any.whl size=3064 sha256=25c827c404cb712aaebb8efaf6b26feb6a76a6c5a4991a884d07c6bdf9cc62fd
  Stored in directory: /root/.cache/pip/wheels/e2/00/23/7328e187ec5202e637b2fb6b2a0c68029758c90abded1eee60
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2

In [8]:
sim_backend = q.Aer.get_backend('qasm_simulator')

In [9]:
KEY_LENGTH = 500
QUANTUM_CHANNEL = []
CLASSICAL_CHANNEL = []

In [10]:
alice_bitstring, alice_bases = select_encoding_and_bases(KEY_LENGTH)
_ , bob_bases = select_encoding_and_bases(KEY_LENGTH)

 
# Preview the first 10 elements of each:
print("Alice randomly generated bitstring: ", alice_bitstring[:10])
print("Alice randomly generated bases: ", alice_bases[:10])

print("Bob randomly generated bases: ", bob_bases[:10])

Alice randomly generated bitstring:  1111000011
Alice randomly generated bases:  0010101000
Bob randomly generated bases:  0010000000


In [11]:
encoded_qubits = encode(alice_bitstring, alice_bases)
QUANTUM_CHANNEL = encoded_qubits

In [12]:
bob_bitstring = measure(bob_bases, QUANTUM_CHANNEL, sim_backend)
print("Bit string at bob's end generated")

Bit string at bob's end generated


In [13]:
CLASSICAL_CHANNEL = alice_bases


In [14]:
agreeing_bases = bob_compare_bases(CLASSICAL_CHANNEL, bob_bases)


In [15]:
CLASSICAL_CHANNEL = agreeing_bases


In [16]:
alice_key = construct_key_from_indices(alice_bitstring, CLASSICAL_CHANNEL)
bob_key = construct_key_from_indices(bob_bitstring, agreeing_bases)

# Preview the first 10 elements of each Key:
print("alice_key: ", alice_key[:10])
print("bob_key:   ", bob_key[:10])
print("Alice's key is equal to Bob's key: ", alice_key == bob_key)
if(not(alice_key == bob_key)):
    print("Thus, this Quantum channel is not safe for data transfer")
elif(alice_key == bob_key):
    print("Thus, this Quantum channel is safe for data transfer")

alice_key:  1111001110
bob_key:    1111001110
Alice's key is equal to Bob's key:  True
Thus, this Quantum channel is safe for data transfer


In [17]:
# QKD Algorithm with evedropping by Eve¶

KEY_LENGTH = 500
QUANTUM_CHANNEL = []
CLASSICAL_CHANNEL = []

In [18]:
alice_bitstring, alice_bases = select_encoding_and_bases(KEY_LENGTH)
_ , bob_bases = select_encoding_and_bases(KEY_LENGTH)
_ , eve_bases = select_encoding_and_bases(KEY_LENGTH)

# Preview the first 10 elements of each:
print("Alice randomly generated bitstring: ", alice_bitstring[:10])
print("Alice randomly generated bases: ", alice_bases[:10])

print("Eve randomly generated bases: ", bob_bases[:10])

print("Bob randomly generated bases: ", eve_bases[:10])

Alice randomly generated bitstring:  0001100001
Alice randomly generated bases:  0110100010
Eve randomly generated bases:  0110101101
Bob randomly generated bases:  1111011010


In [19]:
encoded_qubits = encode(alice_bitstring, alice_bases)
QUANTUM_CHANNEL = encoded_qubits

In [20]:
eve_bitstring = measure(eve_bases, QUANTUM_CHANNEL, sim_backend)
encoded_qubits = encode(eve_bitstring, eve_bases)
QUANTUM_CHANNEL = encoded_qubits
print("Quantum information is measure and encoded by Eve")

Quantum information is measure and encoded by Eve


In [21]:
bob_bitstring = measure(bob_bases, QUANTUM_CHANNEL, sim_backend)
print("Bit string at bob's end generated")

Bit string at bob's end generated


In [22]:
CLASSICAL_CHANNEL = alice_bases


In [23]:
agreeing_bases = bob_compare_bases(CLASSICAL_CHANNEL, bob_bases)


In [24]:
CLASSICAL_CHANNEL = agreeing_bases


In [25]:
alice_key = construct_key_from_indices(alice_bitstring, CLASSICAL_CHANNEL)
bob_key = construct_key_from_indices(bob_bitstring, agreeing_bases)

# Preview the first 10 elements of each Key:
print("alice_key: ", alice_key[:10])
print("bob_key: ", bob_key[:10])
print("Alice's key is equal to Bob's key: ", alice_key == bob_key)
if(not(alice_key == bob_key)):
    print("Thus, this Quantum channel is not safe for data transfer")
elif(alice_key == bob_key):
    print("Thus, this Quantum channel is safe for data transfer")

alice_key:  0001101010
bob_key:  1000111001
Alice's key is equal to Bob's key:  False
Thus, this Quantum channel is not safe for data transfer
